This script will take the sequences in [Gire_Mali_old.fasta](https://github.com/evogytis/EBOV-rates-project-2016/blob/master/alignments/Gire_Mali_old.fasta) and, keeping the sequences from Guinea and Mali intact, sort the labels of the remaining sequences. This will produce a "shuffling" of the sequences, that is, a mismatch between the sequences and their correct labels.

In [ ]:
library(ape)
aln <- read.dna(file = "../alignments/Gire_Mali_old.fasta", format = "fasta")
aln
head(rownames(aln), 10)
( mali.pos <- grep("Mali", rownames(aln)) )
( guinea.pos <- c(grep("Kissidougou", rownames(aln)), grep("Gueckedou", rownames(aln))) )
not.change <- c(mali.pos, guinea.pos)
Tochange <- rownames(aln)[-not.change] ## which labels we will "scramble"
corresp <- data.frame(original = Tochange, shuffled = sort(Tochange)) ## look at the correspondence
head(corresp)
write.table(corresp, file = "../alignments/shuffling_correspondence.txt", row.names = FALSE)
scrambled.Labels <- rownames(aln)
scrambled.Labels[setdiff(seq_along(scrambled.Labels), not.change)] <- sort(Tochange)
aln.scrambled <- aln
rownames(aln.scrambled) <- scrambled.Labels
# Asserting...
all.equal(grep("Mali", rownames(aln)), grep("Mali", rownames(aln.scrambled)))
all.equal(c(grep("Kissidougou", rownames(aln)), grep("Gueckedou", rownames(aln))),
          c(grep("Kissidougou", rownames(aln.scrambled)), grep("Gueckedou", rownames(aln.scrambled)))
)
write.dna(aln.scrambled, file = "../alignments/Gire_Mali_old_shuffled.fasta", format = "fasta")

106 DNA sequences in binary format stored in a matrix.

All sequences of same length: 18959 

Labels: KP260799|Makona-Mali-DPR1|2014-10-23 KP260800|Makona-Mali-DPR2|2014-11-12 KP260801|Makona-Mali-DPR3|2014-11-21 KP260802|Makona-Mali-DPR4|2014-11-12 KM233035|Makona-EM104|2014-06-02 KM233073|Makona-G3786|2014-06-14 ...

More than 1 million nucleotides: not printing base composition

[1] "KP260799|Makona-Mali-DPR1|2014-10-23"
 [2] "KP260800|Makona-Mali-DPR2|2014-11-12"
 [3] "KP260801|Makona-Mali-DPR3|2014-11-21"
 [4] "KP260802|Makona-Mali-DPR4|2014-11-12"
 [5] "KM233035|Makona-EM104|2014-06-02"    
 [6] "KM233073|Makona-G3786|2014-06-14"    
 [7] "KM233078|Makona-G3796|2014-06-15"    
 [8] "KM233042|Makona-EM119|2014-06-03"    
 [9] "KM233040|Makona-EM113|2014-06-03"    
[10] "KM233049|Makona-G3707|2014-06-06"

[1] 1 2 3 4

[1] 104 105 106

,original,shuffled
1,KM233035|Makona-EM104|2014-06-02,KM034549|Makona-EM095B|2014-06-01
2,KM233073|Makona-G3786|2014-06-14,KM034550|Makona-EM095|2014-06-01
3,KM233078|Makona-G3796|2014-06-15,KM034551|Makona-EM096|2014-06-01
4,KM233042|Makona-EM119|2014-06-03,KM034552|Makona-EM098|2014-06-01
5,KM233040|Makona-EM113|2014-06-03,KM034553|Makona-G3670.1|2014-06-01
6,KM233049|Makona-G3707|2014-06-06,KM034554|Makona-G3676.1|2014-06-01


[1] TRUE

[1] TRUE